In [ ]:
# ref: https://www.svm-tutorial.com/2014/10/support-vector-regression-r/

In [1]:
system("ls datasets", intern=TRUE)

[1] "creditcard.csv"      "creditcardfraud.zip"

In [2]:
library(randomForest)
library(e1071)
library(rpart)
library(rpart.plot)
library(caTools)
library(readr)
library(caret)

randomForest 4.6-12
Type rfNews() to see new features/changes/bug fixes.
Loading required package: lattice
Loading required package: ggplot2

Attaching package: ‘ggplot2’

The following object is masked from ‘package:randomForest’:

    margin



In [3]:
card <- read_csv("datasets/creditcard.csv")

Parsed with column specification:
cols(
  .default = col_double(),
  Time = col_integer(),
  Class = col_integer()
)
See spec(...) for full column specifications.
Warning message:
“1 parsing failure.
   row  col               expected actual
153759 Time no trailing characters   e+05
”

In [7]:
# str(card)
card$Class <- factor(card$Class)

In [ ]:
set.seed(1)
split <- sample.split(card$Class, SplitRatio = 0.7)
train <- subset(card, split == T)
cv <- subset(card, split == F)

# check output Class distributiion
table(cv$Class)

In [ ]:
# logistic regression
glm.model <- glm(Class ~ ., data = train, family = "binomial")
glm.predict <- predict(glm.model, cv, type = "response")
table(cv$Class, glm.predict > 0.5)

In [ ]:
# decision tree model

tree.model <- rpart(Class ~ ., data = train, method = "class", minbucket = 20)
prp(tree.model) 
tree.predict <- predict(tree.model, cv, type = "class")
confusionMatrix(cv$Class, tree.predict)

In [ ]:
# Now we only keep 10000 rows of data with class = 0
data.class.0 <- subset(card, card$Class == 0)
data.class.1 <- subset(card, card$Class == 1)
nrow(data.class.0)
nrow(data.class.1)
data.class.0 <- data.class.0[1:10000, ]
nrow(data.class.0)
data <- rbind(data.class.0, data.class.1)
nrow(data)

In [ ]:
# split data 70:30
set.seed(1)
split <- sample.split(data$Class, SplitRatio = 0.7)
train <- subset(data, split == T)
cv <- subset(data, split == F)

table(cv$Class)

In [ ]:
# SVM model
svm.model <- svm(Class ~ ., data = train, kernel = "radial", cost = 1, gamma = 0.1)
svm.predict <- predict(svm.model, cv)
confusionMatrix(cv$Class, svm.predict)

In [ ]:
# Decision Tree Model
tree.model <- rpart(Class ~ ., data = train, method = "class", minbucket = 20)
prp(tree.model) 
tree.predict <- predict(tree.model, cv, type = "class")
confusionMatrix(cv$Class, tree.predict)

In [ ]:
# random forest model
set.seed(10)
rf.model <- randomForest(Class ~ ., data = train,
                         ntree = 2000, nodesize = 20)

rf.predict <- predict(rf.model, cv)
confusionMatrix(cv$Class, rf.predict)
varImpPlot(rf.model)